In [1]:
#Load in the essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations,combinations
from collections import Counter

In [2]:
#create a function that finds pairs and list them in a column. Using A and B as a stand in
def find_pairs(x):
    pairs = pd.DataFrame(list(permutations(x.values,2)),columns=["A","B"])
    return pairs

In [3]:
#Load in the dataset and check the head of the data
dataset=pd.read_csv('Sales_December_2019.csv')
dataset.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


In [4]:
#Group the products by Order ID then apply the function 
dataset_combo =dataset.groupby('Order ID')['Product'].apply(find_pairs).reset_index(drop=True)
dataset_combo.head()

,A,B
0,Google Phone,USB-C Charging Cable
1,Google Phone,Bose SoundSport Headphones
2,Google Phone,Wired Headphones
3,USB-C Charging Cable,Google Phone
4,USB-C Charging Cable,Bose SoundSport Headphones


In [5]:
#  Calculate how often each item item_a occurs with the items in item_b
dataset_combo2 =dataset_combo.groupby(['A','B']).size()
dataset_combo2

A             B                         
20in Monitor  27in FHD Monitor               1
              AA Batteries (4-pack)          1
              AAA Batteries (4-pack)         1
              Apple Airpods Headphones       1
              Bose SoundSport Headphones     1
                                            ..
iPhone        Macbook Pro Laptop             1
              ThinkPad Laptop                1
              USB-C Charging Cable           1
              Vareebadd Phone                1
              Wired Headphones              67
Length: 234, dtype: int64

In [6]:
#create a sorted dataframe by the most frequent combinations.
dataset =dataset_combo2.reset_index()
dataset.columns = ['A','B',"Size"]
dataset.sort_values(by='Size',ascending =False, inplace =True)
dataset.head()

,A,B,Size
166,Product,Product,2256
189,USB-C Charging Cable,Google Phone,122
131,Google Phone,USB-C Charging Cable,122
228,iPhone,Lightning Charging Cable,115
152,Lightning Charging Cable,iPhone,115


In [7]:
#Clean the Product row and check the head of the data
Cleaned_Dataset = dataset[dataset["A"]!="Product"]
Cleaned_Dataset.head()

,A,B,Size
189,USB-C Charging Cable,Google Phone,122
131,Google Phone,USB-C Charging Cable,122
228,iPhone,Lightning Charging Cable,115
152,Lightning Charging Cable,iPhone,115
233,iPhone,Wired Headphones,67


In [8]:
#Create a combination of groups so that can be used as an alternative to pairing
df2=pd.read_csv('Sales_December_2019.csv')
df2= df2.dropna()
df2 =df2[df2['Order ID'].duplicated(keep=False)]
df2['Group'] = df2.groupby('Order ID')['Product'].transform(lambda x:','.join(x))
df2 =df2[['Order ID', 'Group']].drop_duplicates()
df2.head()

,Order ID,Group
16,295681,"Google Phone,USB-C Charging Cable,Bose SoundSp..."
36,295698,"Vareebadd Phone,USB-C Charging Cable"
42,295703,"AA Batteries (4-pack),Bose SoundSport Headphones"
66,295726,"iPhone,Lightning Charging Cable"
76,295735,"iPhone,Apple Airpods Headphones,Wired Headphones"


In [9]:
#Count each row combination by the the pairing of the two
count = Counter()
for row in df2['Group']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))

print(count)

Counter({('Product', 'Product'): 1128, ('Google Phone', 'USB-C Charging Cable'): 118, ('iPhone', 'Lightning Charging Cable'): 113, ('iPhone', 'Wired Headphones'): 66, ('Google Phone', 'Wired Headphones'): 56, ('Vareebadd Phone', 'USB-C Charging Cable'): 53, ('iPhone', 'Apple Airpods Headphones'): 52, ('Google Phone', 'Bose SoundSport Headphones'): 27, ('USB-C Charging Cable', 'Wired Headphones'): 25, ('Vareebadd Phone', 'Wired Headphones'): 22, ('Apple Airpods Headphones', 'Wired Headphones'): 13, ('USB-C Charging Cable', 'Lightning Charging Cable'): 11, ('Lightning Charging Cable', 'Wired Headphones'): 11, ('Lightning Charging Cable', 'AA Batteries (4-pack)'): 11, ('Lightning Charging Cable', 'USB-C Charging Cable'): 11, ('AA Batteries (4-pack)', 'Lightning Charging Cable'): 10, ('Vareebadd Phone', 'Bose SoundSport Headphones'): 10, ('Bose SoundSport Headphones', 'Wired Headphones'): 9, ('Wired Headphones', 'Wired Headphones'): 9, ('AA Batteries (4-pack)', 'AA Batteries (4-pack)'): 9,

In [10]:
#check out the most common combination. 
for key, value in count.most_common(10):
    print(key,value)


('Product', 'Product') 1128
('Google Phone', 'USB-C Charging Cable') 118
('iPhone', 'Lightning Charging Cable') 113
('iPhone', 'Wired Headphones') 66
('Google Phone', 'Wired Headphones') 56
('Vareebadd Phone', 'USB-C Charging Cable') 53
('iPhone', 'Apple Airpods Headphones') 52
('Google Phone', 'Bose SoundSport Headphones') 27
('USB-C Charging Cable', 'Wired Headphones') 25
('Vareebadd Phone', 'Wired Headphones') 22
